# Dataset Validation Testing

This notebook uses a dataset [here](https://kilthub.cmu.edu/articles/dataset/eVTOL_Battery_Dataset/14226830) to validate the model.

In [ ]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import zipfile
import requests
import progressbar

In [ ]:
# start by downloading and extracting the dataset if not already downloaded
dataset_link = 'https://kilthub.cmu.edu/ndownloader/articles/14226830/versions/1'
filename = 'res/14226830.zip'
dataset_path = 'res/dataset'
if not os.path.exists(filename):
    print('Downloading dataset...')
    with requests.get(dataset_link, stream=True) as r:
        total_len = int(r.headers.get('Content-Length'))
        with open(filename, 'wb') as f:
            bar = progressbar.ProgressBar(max_value=total_len)
            for chunk in r.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
                    bar.update(f.tell())
            bar.finish()
    print('Download complete.')
else:
    print('Dataset already downloaded.')
if not os.path.exists(dataset_path):
    print('Extracting dataset...')
    os.mkdir(dataset_path)
    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall(dataset_path)
    print('Extraction complete.')
else:
    print('Dataset already extracted.')

The dataset csv files are as follows:

| File Name | Description |
| --------- | ----------- |
|VAH01.csv| Baseline 
|VAH02.csv| Extended cruise (1000 sec)|
|VAH05.csv| 10% power reduction during discharge (flight)|
|VAH06.csv| CC charge current reduced to C/2|
|VAH07.csv| CV charge voltage reduced to 4.0V|
|VAH09.csv| Thermal chamber temperature of 20 degrees C|
|VAH10.csv| Thermal chamber temperature of 30 degrees C|
|VAH11.csv| 20% power reduction during discharge (flight)|
|VAH12.csv| Short cruise length (400 sec)|
|VAH13.csv| Short cruise length (600 sec)|
|VAH15.csv| Extended cruise (1000 sec)|
|VAH16.csv| CC charge current reduced to 1.5C|
|VAH17.csv| Baseline|
|VAH20.csv| Charge current reduced to 1.5C|
|VAH22.csv| Extended cruise (1000 sec)|
|VAH23.csv| CV charge voltage reduced to 4.1V|
|VAH24.csv| CC charge current reduced to C/2|
|VAH25.csv| Thermal chamber temperature of 20 degrees C|
|VAH26.csv| Short cruise length (600 sec)|
|VAH27.csv| Baseline|
|VAH28.csv| 10% power reduction during discharge (flight)|
|VAH30.csv| Thermal chamber temperature of 35 degrees C |

Each dataset has the following columns:

| Column Name | Description |
| ----------- | ----------- |
| time_s | Time since beginning of experiment in seconds|
| Ecell_V | Cell Voltage|
| I_mA | Cell current in milliamperes|
| EnergyCharge_W_h | The amount of energy supplied to the cell during charge in watt-hours|
| QCharge_mA_h | The amout of charge supplied to the cell during charge in milliampere-hours|
| EnergyDischarge_W_h | The amount of energy extracted from the cell during discharge in watt-hours|
| QDischarge_mA_h | The amount of charge extracted from the cell during discharge in milliampere-hours|
| Temperature__C | Cell surface temperature in degrees celcius|
| cycleNumber | Cycle number as recorded by the cell tester (see above note)|
| Ns | Cycle segment (varies, useful for discerning between segments) |

In [ ]:
# import the data into dataframes
print('Importing data...')
df = pd.read_csv(os.path.join(dataset_path, 'VAH01.csv'))
print('Import complete.')
# truncate the dataframe so I can do calculations in a reasonable amount of time
df = df.iloc[:30000,:]

In [ ]:
# plot voltage, current, and power over time
print('Plotting data...')
fig, ax = plt.subplots(3, 1, figsize=(10, 10))
ax[0].plot(df['time_s'], df['Ecell_V'])
ax[0].set_title('Voltage')
ax[1].plot(df['time_s'], df['I_mA'])
ax[1].set_title('Current')
ax[2].plot(df['time_s'], df['QDischarge_mA_h'], label='Discharge')
ax[2].plot(df['time_s'], df['QCharge_mA_h'], label='Charge')
ax[2].set_title('Charge (mAh)')
ax[2].legend()
plt.show()
